In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time

In [ ]:
def calculate_image_brightness(image):
    """Calculate average brightness of an image"""
    try:
        # Convert to grayscale if the image is in color
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image
        
        # Calculate average brightness
        return np.mean(gray)
    except Exception as e:
        print(f"Error calculating image brightness: {e}")
        return 0

In [ ]:
from picamera2 import Picamera2
import libcamera
picam2 = Picamera2()
frame_size = (4608, 2592)

# Get initial exposure time based on time of day

config = picam2.create_still_configuration(
    {"size": frame_size},
    controls={"ExposureTime": 50000},
    transform=libcamera.Transform(vflip=1),
)
picam2.configure(config)
picam2.start()


In [ ]:
def calculate_image_quality(image):
    """Calculate image quality metrics including brightness, contrast, and histogram distribution"""
    try:
        # Convert to grayscale if the image is in color
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image
        
        # Calculate average brightness
        avg_brightness = np.mean(gray)
        
        # Calculate standard deviation (simple contrast measure)
        std_dev = np.std(gray)
        
        # Calculate histogram
        hist = cv2.calcHist([gray], [0], None, [256], [0, 256])
        hist = hist.flatten() / hist.sum()  # Normalize histogram
        
        # Calculate histogram standard deviation (measure of spread)
        hist_indices = np.arange(256)
        hist_mean = np.sum(hist_indices * hist)
        hist_std = np.sqrt(np.sum(((hist_indices - hist_mean) ** 2) * hist))
        
        # Calculate contrast ratio (simplified)
        p_low = np.percentile(gray, 5)  # 5th percentile
        p_high = np.percentile(gray, 95)  # 95th percentile
        if p_low > 0:  # Avoid division by zero
            contrast_ratio = p_high / p_low
        else:
            contrast_ratio = p_high
        
        return {
            'avg_brightness': avg_brightness,
            'std_dev': std_dev,
            'hist_std': hist_std,
            'contrast_ratio': contrast_ratio
        }
    except Exception as e:
        print(f"Error calculating image quality: {e}")
        return {
            'avg_brightness': 0,
            'std_dev': 0,
            'hist_std': 0,
            'contrast_ratio': 0
        }

In [ ]:
desired_brightness = 0.5  # Set desired brightness level (0-1)
exposure_tolerance = 50  # Set tolerance for brightness adjustment

picam2.set_controls({'ExposureTime': 5000})

def set_exposure_time(exposure_time):
    """Set the exposure time for the camera"""
    desired_exposure_time = exposure_time  # Set desired exposure time in microseconds
    exposure_tolerance = 50  # Set tolerance for exposure adjustment
    
    picam2.set_controls({'ExposureTime': exposure_time})

    for i in range(3):
        actual_exposure = picam2.get_controls()['ExposureTime']
        if abs(actual_exposure - desired_exposure_time) <= exposure_tolerance:
            break

while True:
    frame = picam2.capture_array()
    brightness = calculate_image_brightness(frame) / 255.0  # Normalize brightness to [0, 1]

    if abs(brightness - desired_brightness) <= exposure_tolerance:
        break

    # Adjust exposure time based on brightness
    if brightness < desired_brightness:
        set_exposure_time(picam2.get_controls()['ExposureTime'] + 2000)
    else:
        set_exposure_time(picam2.get_controls()['ExposureTime'] - 2000)



plt.imshow(frame)
plt.axis('off')

